In [1]:
# importations
import numpy as np
import pandas as pd
from pyedflib import EdfReader
import mne
import EntropyHub

import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from scipy.interpolate import PchipInterpolator,interp1d

%matplotlib inline

import copy
import os
import importlib

import generisAPI
importlib.reload(generisAPI)

import generisAPI.file_operators as fp
importlib.reload(generisAPI.file_operators)

import generisAPI.data_cleaning as dcln
importlib.reload(generisAPI.data_cleaning)

import generisAPI.preprocessing as ppcn
importlib.reload(generisAPI.preprocessing)

import generisAPI.processing as pcn
importlib.reload(generisAPI.processing)

import generisAPI.audProcessing as audpcn
importlib.reload(generisAPI.audProcessing)

import generisAPI.plots as plots
importlib.reload(generisAPI.plots)

<module 'generisAPI.plots' from 'd:\\Emmanuel_petron_Olateju\\MMN_SZ\\notebooks\\generisAPI\\plots.py'>

### Import, compile data and preprocess

In [2]:
# definitions
electrodes=[
    'Fp1','Fp2','F3','F4','C3',
    'C4','P3','P4','O1','O2',
    'F7','F8','T3','T4','T5',
    'T6','Fz','Pz','Cz',
    ]
len(electrodes)

19

In [3]:
# compile eeg_data and annotations
subjects_dataset_root_path = '../acquired_dataset/'

data = fp.xtract_id_subject_info_dict(subjects_dataset_root_path)
print(len(data))

32


### Preprocessing

In [4]:
# assert eeg_data trials match marker trials

trialMarkerLength = dcln.assertTrialsMarkersLength()
trialMarkerLength.fit(data)
print(trialMarkerLength.indices)
clean_data = trialMarkerLength.transform(data,False)

[]


In [5]:
# check for null eeg_data

null_data_checker = dcln.nullIndicesTracer()
null_data_checker.fit(clean_data)
print(null_data_checker.indices)
clean_data = null_data_checker.transform(clean_data,True)

{'16': {'rest1': [0, 1], 'arith': [0, 1], 'rest2': [0, 1], 'auditory': [0, 1]}, '20': {'rest1': [], 'arith': [], 'rest2': [], 'auditory': [0]}, '23': {'rest1': [], 'arith': [0], 'rest2': [0], 'auditory': [0]}, '32': {'rest1': [0, 1, 2], 'arith': [0, 1, 2], 'rest2': [0, 1, 2], 'auditory': [0, 1, 2]}, '4': {'rest1': [], 'arith': [], 'rest2': [2], 'auditory': [2]}, '5': {'rest1': [0, 1], 'arith': [0, 1], 'rest2': [0, 1], 'auditory': [0, 1]}, '6': {'rest1': [], 'arith': [], 'rest2': [], 'auditory': [0]}, '8': {'rest1': [0, 1], 'arith': [0, 1], 'rest2': [0, 1], 'auditory': [0, 1]}, '9': {'rest1': [0, 1], 'arith': [0, 1], 'rest2': [0, 1], 'auditory': [0, 1]}}


In [6]:
preprocess_method=[
    [ppcn.channel_dropper,{'index':[19,20,21,22,23],'axis':0}],
    # [ppcn.filter,{'lfreq':100,'hfreq':None,'sfreq':200}],
    # [ppcn.filter,{'lfreq':75,'hfreq':45,'sfreq':200}],
    # [ppcn.resampler,{'up':1,'down':1}],
    [ppcn.baseline_corrector,{'with_std':False}]
]

preprocess_pipeline = ppcn.ppcn_pipeline(preprocess_method)

mne.set_log_level('ERROR')
clean_data = ppcn.all_subjects_preprocessor(
    preprocess_pipeline,
    clean_data
)

In [7]:
auditory_data = pcn.xtract_phase_data('auditory',clean_data)
arithmetic_data = pcn.xtract_phase_data('arith',clean_data)
firstRest_data = pcn.xtract_phase_data('rest1',clean_data)
secondRest_data = pcn.xtract_phase_data('rest2',clean_data)

### Auditory stimuli processing

In [8]:
aud_data = audpcn.all_subjects_phase_auditory_epochs(auditory_data,0.1)

In [9]:
aud_method=[
    [audpcn.aud_stimuli_trial_average,{}],
    [ppcn.electrode_grouper,{'electrodes':electrodes,'target':'T'}]
]

aud_pcn_pipeline = audpcn.pipeline(aud_method)

avg_aud = audpcn.all_subjects_processor(aud_pcn_pipeline,aud_data)

### STFT Feature

In [10]:
stft_feature_method=[
    [pcn.trial_epoching,{'dummy':0}],
    [pcn.stft,{'fs':9,'sfreq':200}],
    [pcn.trial_averaging,{'axis':0,'dummy':0}],
    [ppcn.electrode_grouper,{'electrodes':electrodes,'target':'F'}],
    [pcn.trial_averaging,{'axis':0,'dummy':0}],
]

stft_feature_pipeline=ppcn.ppcn_pipeline(stft_feature_method)

firstRest_stft = pcn.all_subjects(stft_feature_pipeline,firstRest_data)
arithmetic_stft = pcn.all_subjects(stft_feature_pipeline,arithmetic_data)
secondRest_stft = pcn.all_subjects(stft_feature_pipeline,secondRest_data)
auditory_stft = pcn.all_subjects(stft_feature_pipeline,auditory_data)

d:\Emmanuel_petron_Olateju\MMN_SZ\notebooks\generisAPI\processing.py:52: ComplexWarning: Casting complex values to real discards the imaginary part
  res[i,:,:,:] = self.__stft(X[i,:,:])


1
10
11
12
13
14
15
16
17
18
19
2
20
21
22
23
24
25
26
27
28
29
3
30
31
32
4
5
6
7
8
9
1
10
11
12
13
14
15
16
17
18
19
2
20
21
22
23
24
25
26
27
28
29
3
30
31
32
4
5
6
7
8
9
1
10
11
12
13
14
15
16
17
18
19
2
20
21
22
23
24
25
26
27
28
29
3
30
31
32
4
5
6
7
8
9
1
10
11
12
13
14
15
16
17
18
19
2
20
21
22
23
24
25
26
27
28
29
3
30
31
32
4
5
6
7
8
9


### FuzzyEntropy Feature

In [11]:
# frontal electrodes
fuzzy_entropy_method=[
    [ppcn.electrode_grouper,{'electrodes':electrodes,'target':'F'}],
    [pcn.trial_epoching,{'dummy':0}],
    [pcn.trial_averaging,{}],
    [pcn.fuzzEnt,{'m':2}]
]

fuzzy_entropy_pipeline=ppcn.ppcn_pipeline(fuzzy_entropy_method)

frontal_entropies={
    'rest1':pcn.all_subjects(fuzzy_entropy_pipeline,firstRest_data),
    'arith':pcn.all_subjects(fuzzy_entropy_pipeline,arithmetic_data),
    'rest2':pcn.all_subjects(fuzzy_entropy_pipeline,secondRest_data),
    'auditory':pcn.all_subjects(fuzzy_entropy_pipeline,auditory_data)
}

1
10
11
12
13
14
15
16
17
18
19
2
20
21
22
23
24
25
26
27
28
29
3
30
31
32
4
5
6
7
8
9
1
10
11
12
13
14
15
16
17
18
19
2
20
21
22
23
24
25
26
27
28
29
3
30
31
32
4
5
6
7
8
9
1
10
11
12
13
14
15
16
17
18
19
2
20
21
22
23
24
25
26
27
28
29
3
30
31
32
4
5
6
7
8
9
1
10
11
12
13
14
15
16
17
18
19
2
20
21
22
23
24
25
26
27
28
29
3
30
31
32
4
5
6
7
8
9


In [12]:
# parietal and occipital electrodes
fuzzy_entropy_method=[
    [ppcn.electrode_grouper,{'electrodes':electrodes,'target':['P','T']}],
    [pcn.trial_epoching,{'dummy':0}],
    [pcn.trial_averaging,{}],
    [pcn.fuzzEnt,{'m':2}]
]

fuzzy_entropy_pipeline=ppcn.ppcn_pipeline(fuzzy_entropy_method)

parietal_occipital_entropies={
    'rest1':pcn.all_subjects(fuzzy_entropy_pipeline,firstRest_data),
    'arith':pcn.all_subjects(fuzzy_entropy_pipeline,arithmetic_data),
    'rest2':pcn.all_subjects(fuzzy_entropy_pipeline,secondRest_data),
    'auditory':pcn.all_subjects(fuzzy_entropy_pipeline,auditory_data)
}

1
10
11
12
13
14
15
16
17
18
19
2
20
21
22
23
24
25
26
27
28
29
3
30
31
32
4
5
6
7
8
9
1
10
11
12
13
14
15
16
17
18
19
2
20
21
22
23
24
25
26
27
28
29
3
30
31
32
4
5
6
7
8
9
1
10
11
12
13
14
15
16
17
18
19
2
20
21
22
23
24
25
26
27
28
29
3
30
31
32
4
5
6
7
8
9
1
10
11
12
13
14
15
16
17
18
19
2
20
21
22
23
24
25
26
27
28
29
3
30
31
32
4
5
6
7
8
9


In [13]:
# temporal_occipital electrodes
fuzzy_entropy_method=[
    [ppcn.electrode_grouper,{'electrodes':electrodes,'target':['T','O']}],
    [pcn.trial_epoching,{'dummy':0}],
    [pcn.trial_averaging,{}],
    [pcn.fuzzEnt,{'m':2}]
]

fuzzy_entropy_pipeline=ppcn.ppcn_pipeline(fuzzy_entropy_method)

temporal_occipital_entropies={
    'rest1':pcn.all_subjects(fuzzy_entropy_pipeline,firstRest_data),
    'arith':pcn.all_subjects(fuzzy_entropy_pipeline,arithmetic_data),
    'rest2':pcn.all_subjects(fuzzy_entropy_pipeline,secondRest_data),
    'auditory':pcn.all_subjects(fuzzy_entropy_pipeline,auditory_data)
}

1
10
11
12
13
14
15
16
17
18
19
2
20
21
22
23
24
25
26
27
28
29
3
30
31
32
4
5
6
7
8
9
1
10
11
12
13
14
15
16
17


error uploading: HTTPSConnectionPool(host='api.segment.io', port=443): Max retries exceeded with url: /v1/batch (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001E08DE5C8B0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))


18
19
2
20
21
22
23
24
25
26
27
28
29
3
30
31
32
4
5
6
7
8
9
1
10
11
12
13
14
15
16
17
18
19
2
20
21
22
23
24
25
26
27
28
29
3
30
31
32
4
5
6
7
8
9
1
10
11
12
13
14
15
16
17
18
19
2
20
21
22
23
24
25
26
27
28
29
3
30
31
32
4
5
6
7
8
9


In [14]:
# parietal and central electrodes
fuzzy_entropy_method=[
    [ppcn.electrode_grouper,{'electrodes':electrodes,'target':['C','P']}],
    [pcn.trial_epoching,{'dummy':0}],
    [pcn.trial_averaging,{}],
    [pcn.fuzzEnt,{'m':2}]
]

fuzzy_entropy_pipeline=ppcn.ppcn_pipeline(fuzzy_entropy_method)

parietal_central_entropies={
    'rest1':pcn.all_subjects(fuzzy_entropy_pipeline,firstRest_data),
    'arith':pcn.all_subjects(fuzzy_entropy_pipeline,arithmetic_data),
    'rest2':pcn.all_subjects(fuzzy_entropy_pipeline,secondRest_data),
    'auditory':pcn.all_subjects(fuzzy_entropy_pipeline,auditory_data)
}

1
10
11
12
13
14
15
16
17
18
19
2
20
21
22
23
24
25
26
27
28
29
3
30
31
32
4
5
6
7
8
9
1
10
11
12
13
14
15
16
17
18
19
2
20
21
22
23
24
25
26
27
28
29
3
30
31
32
4
5
6
7
8
9
1
10
11
12
13
14
15
16
17
18
19
2
20
21
22
23
24
25
26
27
28
29
3
30
31
32
4
5
6
7
8
9
1
10
11
12
13
14
15
16
17
18
19
2
20
21
22
23
24
25
26
27
28
29
3
30
31
32
4
5
6
7
8
9


### Plots

In [36]:
for s in data:
    if s not in null_data_checker.indices:
        fig,ax = plt.subplots(3,4,figsize=(12,7))
        plots.mmn_plot(ax[0,0:4],avg_aud[s])
        plots.stft_plot(ax[1,0],firstRest_stft[s][0],'1st Rest')
        plots.stft_plot(ax[1,1],arithmetic_stft[s][0],'Arithmetic Task')
        plots.stft_plot(ax[1,2],secondRest_stft[s][0],'2nd Rest')
        plots.stft_plot(ax[1,3],auditory_stft[s][0],'Auditory Task')
        plots.entropy_plot(ax[2,0],frontal_entropies['rest1'][s],1)
        plots.entropy_plot(ax[2,1],frontal_entropies['arith'][s],1)
        plots.entropy_plot(ax[2,2],frontal_entropies['rest2'][s],1)
        plots.entropy_plot(ax[2,3],frontal_entropies['auditory'][s],1)
        plots.entropy_plot(ax[2,0],parietal_central_entropies['rest1'][s],2)
        plots.entropy_plot(ax[2,1],parietal_central_entropies['arith'][s],2)
        plots.entropy_plot(ax[2,2],parietal_central_entropies['rest2'][s],2)
        plots.entropy_plot(ax[2,3],parietal_central_entropies['auditory'][s],2)
        plots.entropy_plot(ax[2,0],parietal_occipital_entropies['rest1'][s],3)
        plots.entropy_plot(ax[2,1],parietal_occipital_entropies['arith'][s],3)
        plots.entropy_plot(ax[2,2],parietal_occipital_entropies['rest2'][s],3)
        plots.entropy_plot(ax[2,3],parietal_occipital_entropies['auditory'][s],3)
        plots.entropy_plot(ax[2,0],temporal_occipital_entropies['rest1'][s],4)
        plots.entropy_plot(ax[2,1],temporal_occipital_entropies['arith'][s],4)
        plots.entropy_plot(ax[2,2],temporal_occipital_entropies['rest2'][s],4)
        plots.entropy_plot(ax[2,3],temporal_occipital_entropies['auditory'][s],4)
        fig.suptitle(s+':'+data[s]['category']+'_'+data[s]['name']+'_'+data[s]['sex']+'_'+str(data[s]['age']))
        fig.tight_layout(pad=1.5)
        # plt.subplots_adjust(left=0, bottom=0, right=1, top=1, wspace=0, hspace=0)
        x0, x1, y0, y1 = plt.axis()
        # plt.axis((
        #     x0+0,
        #     x1+6,
        #     y0+0,
        #     y1+0
        # ))
        plt.ioff()
        plt.savefig('../data_analysis_results/results/'+data[s]['category']+'/'+s+'.png',format='png')